<a href="https://colab.research.google.com/github/fdeiab/loan-default-prediction/blob/main/Loan_Default_Predicition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

Loan approval is a critical decision for any lending institution. One of two scenarios may happen, the bank loses out on potential income by rejecting a loan to an individual or a company. Or, they lend a loan to a party exhibiting risky behaviour where heavy losses. This uncertainty is why it is a good idea that banks utilize good loan default prediction models as they stand much to gain. 

**The major goal of this notebook is to assess if a loan should be approved and predict whether a loan will default.**

# The Data

The dataset used in this project, "SBA Case", is a subset of the "National SBA" data, which contains historical data from 1987 through 2014. The subset dataset focuses solely on the state of California through the years of 1989 to 2012. The dataset used contains **2,102** records under **35** attributes:


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

df = pd.read_csv('/content/drive/MyDrive/SBAcase.11.13.17.csv')
metadata_1 = pd.read_csv('/content/drive/MyDrive/metadata1.csv')
metadata_2 = pd.read_csv('/content/drive/MyDrive/metadata2.csv')
metadata = pd.concat([metadata_1, metadata_2])
metadata = metadata.drop(columns='Data type')

print(metadata.to_markdown(tablefmt="fancy_grid", index=False))



╒═══════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Variable name     │ Description of variable                                                                                                                   │
╞═══════════════════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ LoanNr_ChkDgt     │ Identifier – Primary key                                                                                                                  │
├───────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Name              │ Borrower name                                                                                                                             │
├───────────────────┼───────